# Exploratory Data Analysis (EDA) of the sensory data

In [1]:
import pandas as pd
import numpy as np

# Load the data
data = pd.read_csv('../data/all_data.csv')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [6]:
# Show some basic information
print(data.columns)
print(data.info())
data.describe()
print(data.columns)


Index(['Time (s)', 'Latitude (°)', 'Longitude (°)', 'Height (m)',
       'Velocity (m/s)', 'Direction (°)', 'Horizontal Accuracy (m)',
       'Vertical Accuracy (°)', 'X (µT)', 'Y (µT)', 'Z (µT)', 'Distance (cm)',
       'timestamp', 'transportation_mode', 'start_date', 'expirement_id',
       'Acceleration x (m/s^2)', 'Acceleration y (m/s^2)',
       'Acceleration z (m/s^2)', 'Gyroscope x (rad/s)', 'Gyroscope y (rad/s)',
       'Gyroscope z (rad/s)', 'Illuminance (lx)',
       'Linear Acceleration x (m/s^2)', 'Linear Acceleration y (m/s^2)',
       'Linear Acceleration z (m/s^2)', 'Vertical Accuracy (m)',
       'Magnetic field x (µT)', 'Magnetic field y (µT)',
       'Magnetic field z (µT)', 'Pressure (hPa)', 'Temperature (°C)'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385400 entries, 0 to 385399
Data columns (total 32 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0 

In [7]:
# Find amount of null values
print(data.isnull().sum())

Time (s)                              0
Latitude (°)                     378152
Longitude (°)                    378152
Height (m)                       378213
Velocity (m/s)                   378268
Direction (°)                    378420
Horizontal Accuracy (m)          378152
Vertical Accuracy (°)            380041
X (µT)                           329228
Y (µT)                           329228
Z (µT)                           329228
Distance (cm)                    379839
timestamp                             0
transportation_mode                   0
start_date                            0
expirement_id                         0
Acceleration x (m/s^2)           298095
Acceleration y (m/s^2)           298095
Acceleration z (m/s^2)           298095
Gyroscope x (rad/s)              298099
Gyroscope y (rad/s)              298099
Gyroscope z (rad/s)              298099
Illuminance (lx)                 368093
Linear Acceleration x (m/s^2)    354257
Linear Acceleration y (m/s^2)    354257


# Create some graphs of gyroscope data per transportation_mode, as well as the acceleration data
import matplotlib.pyplot as plt
import seaborn as sns

# Gyroscope data
plt.figure(figsize=(10, 6))
sns.boxplot(x='transportation_mode', y='Gyroscope x (rad/s)', data=data)
plt.savefig('gyro_x.png')
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='transportation_mode', y='Gyroscope y (rad/s)', data=data)
plt.savefig('gyro_y.png')
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='transportation_mode', y='Gyroscope z (rad/s)', data=data)
plt.savefig('gyro_z.png')
plt.show()

# Acceleration data
plt.figure(figsize=(10, 6))
sns.boxplot(x='transportation_mode', y='Acceleration x (m/s^2)', data=data)
plt.savefig('accel_x.png')
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='transportation_mode', y='Acceleration y (m/s^2)', data=data)
plt.savefig('accel_y.png')
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='transportation_mode', y='Acceleration z (m/s^2)', data=data)
plt.savefig('accel_z.png')
plt.show()

plt.figure(figsize=(10,6))
sns.lineplot(x=, y='Height (m)', data=data)
plt.savefig('height.png')
plt.show()


